### ドライブのマウント

In [1]:
# Googleドライブをマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### ライブラリ・モジュールのインポート

In [2]:
# ライブラリの準備
!pip install timm
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn.functional as F
import torch.autograd as autograd
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from timm.scheduler import CosineLRScheduler
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import copy
import random
import pickle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 25.3 MB/s eta 0:00:00


### シード値の設定

In [3]:
# シード値を設定
def fix_seed(seed=1234):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

fix_seed(seed=1234)

### データセットの準備

In [4]:
# 平均値と標準偏差を計算するためのCIFAR10データセットの前処理を定義
pre_transform_cifar10 = transforms.Compose([
    transforms.ToTensor()
])

# 平均値と標準偏差を計算するためのCIFAR10データセットの読み込み
pre_train_dataset_cifar10 = datasets.CIFAR10(root='/content/data/', download=True, transform=pre_transform_cifar10)

# 平均値と標準偏差を計算するための変数を初期化
pre_mean_cifar10 = 0.0
pre_std_cifar10 = 0.0
pre_total_samples_cifar10 = len(pre_train_dataset_cifar10)

# データセットのすべてのデータポイントに対して平均値と標準偏差を計算
for data in pre_train_dataset_cifar10:
    pre_image, _ = data
    pre_mean_cifar10 += pre_image.mean(dim=(1, 2))  # テンソルのチャンネルごとに平均を計算
    pre_std_cifar10 += pre_image.std(dim=(1, 2))    # テンソルのチャンネルごとに標準偏差を計算

# データセット全体の平均値と標準偏差を計算
pre_mean_cifar10 /= pre_total_samples_cifar10
pre_std_cifar10 /= pre_total_samples_cifar10

print("データセット全体の平均値: ", pre_mean_cifar10)
print("データセット全体の標準偏差: ", pre_std_cifar10)

100%|██████████| 170498071/170498071 [00:05<00:00, 32354643.53it/s]


Extracting /content/data/cifar-10-python.tar.gz to /content/data/
データセット全体の平均値:  tensor([0.4914, 0.4822, 0.4465])
データセット全体の標準偏差:  tensor([0.2023, 0.1994, 0.2010])


In [5]:
# 学習用のCIFAR10データセットの前処理を定義
train_transform_cifar10 = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=pre_mean_cifar10, std=pre_std_cifar10)
    ])
# テスト用のCIFAR10データセットの前処理を定義
test_transform_cifar10 = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=pre_mean_cifar10, std=pre_std_cifar10)
    ])

# 学習用のCIFAR10データセットの読み込み
train_dataset_cifar10 = torchvision.datasets.CIFAR10(root='/content/data/', train=True, transform=train_transform_cifar10, download=True)
# テスト用のCIFAR10データセットの読み込み
test_dataset_cifar10 = torchvision.datasets.CIFAR10(root='/content/data/', train=False, transform=test_transform_cifar10, download=True)

# 学習用のCIFAR10データローダーを作成
train_loader_cifar10 = torch.utils.data.DataLoader(dataset=train_dataset_cifar10, batch_size=512, shuffle=True, num_workers=2)
# テスト用のCIFAR10データローダーを作成
test_loader_cifar10 = torch.utils.data.DataLoader(dataset=test_dataset_cifar10, batch_size=512, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


## モデルの実装

In [6]:
# 1×1のサブネットワーク獲得用の畳み込みを定義
def supermaskconv1x1(in_channels, out_channels, stride=1):
    return SupermaskConv(in_channels, out_channels, kernel_size=1, stride=stride, bias=False)

# 3×3のサブネットワーク獲得用の畳み込みを定義
def supermaskconv3x3(in_channels, out_channels, stride=1):
    return SupermaskConv(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)

In [7]:
# サブネットワーク獲得用のバッチ正則化として、非アフィン正規化を使用する（学習可能なパラメータを使用しない）
class NonAffineBatchNorm(nn.BatchNorm2d):
    def __init__(self, dim):
        super(NonAffineBatchNorm, self).__init__(dim, affine=False)

In [8]:
# 各重みにスコアを付与したスコアをソートしてtop k%を使用することでサブネットワークを獲得する
class GetSubnet(autograd.Function):
    @staticmethod
    def forward(ctx, scores, k):
        # スコアを複製する
        out = scores.clone()
        # スコアを昇順でソートする
        _, idx = scores.flatten().sort()
        # top k%以下のスコアの数
        j = int((1 - k) * scores.numel())
        # flat_outとoutは同じメモリを参照する（flat_outを変更するとoutにも影響する）
        flat_out = out.flatten()
        # top k%の要素を1にする
        flat_out[idx[j:]] = 1
        # top k%以外の要素を0にする
        flat_out[idx[:j]] = 0
        return out

    @staticmethod
    def backward(ctx, g):
        # 逆伝播時に勾配gをそのまま伝える
        return g, None

In [9]:
# サブネットワーク獲得用の畳み込みを定義
class SupermaskConv(nn.Conv2d):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        # 重みと同じ形状のスコアを用意
        self.scores = nn.Parameter(torch.Tensor(self.weight.size()))
        # スコアを一様分布で初期化
        nn.init.kaiming_uniform_(self.scores, a=math.sqrt(5))
        # 重みの勾配を無効化する
        self.weight.requires_grad = False

    def _init_conv(self):
        # 重みを一様分布で初期化
        if self.init == 'kaiming_uniform':
            nn.init.kaiming_uniform_(self.weight, mode='fan_out', nonlinearity='relu')
        # 重みを正規分布で初期化
        elif self.init == 'kaiming_normal':
            nn.init.kaiming_normal_(self.weight, mode='fan_out', nonlinearity='relu')
        # 重みを符号つき定数で初期化
        elif self.init == 'signed_constant':
            fan = nn.init._calculate_correct_fan(self.weight, mode='fan_out')
            gain = nn.init.calculate_gain('relu')
            std = gain / math.sqrt(fan)
            self.weight.data = self.weight.data.sign() * std

    def set_init(self, init):
        # 初期化手法を設定
        self.init = init

    def set_prune_rate(self, prune_rate):
        # 刈り込み率を設定
        self.prune_rate = prune_rate

    @property
    def clamped_scores(self):
        # スコアとして非負の値を返すようにする（重要度を表す）
        return self.scores.abs()

    def forward(self, x):
        # サブネットワークを獲得
        subnet = GetSubnet.apply(self.clamped_scores, 1 - self.prune_rate)
        # サブネットワークでマスク
        w = self.weight * subnet
        x = F.conv2d(x, w, self.bias, self.stride, self.padding, self.dilation, self.groups)
        return x

In [10]:
# サブネットワーク獲得用のResidual Blocksを定義
class SupermaskBasicBlock(nn.Module):
    # 入力チャンネル数に対する出力チャンネル数の倍率
    expansion = 1

    def __init__(self, in_channels, channels, stride=1):
        super().__init__()
        self.conv1 = supermaskconv3x3(in_channels, channels, stride)
        self.bn1 = NonAffineBatchNorm(channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = supermaskconv3x3(channels, channels)
        self.bn2 = NonAffineBatchNorm(channels)
        # 入力と出力のチャンネル数が異なる場合（strideが1より大きい場合）、ダウンサンプリング
        if in_channels != channels * self.expansion or stride > 1:
            self.shortcut = nn.Sequential(
                supermaskconv1x1(in_channels, channels * self.expansion, stride),
                NonAffineBatchNorm(channels * self.expansion)
            )
        else:
            self.shortcut = nn.Sequential()

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        # 残差写像と恒等写像の和を計算
        out += self.shortcut(x)
        out = self.relu(out)
        return out

In [11]:
# サブネットワーク獲得用のResNetを定義
class SupermaskResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super().__init__()
        self.in_channels = 64
        self.conv1 = SupermaskConv(3, self.in_channels, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = NonAffineBatchNorm(self.in_channels)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        # Residual Blocks(in_channels=64)
        self.layer1 = self._make_layer(block, 64, layers[0], stride=1)
        # Residual Blocks(in_channels=128)
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        # Residual Blocks(in_channels=256)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        # Residual Blocks(in_channels=512)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        # 重みの初期化
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                # 初期化手法を設定
                m.set_init(init)
                # 刈り込み率を設定
                m.set_prune_rate(prune_rate)
                # 重みを初期化
                m._init_conv()

    # Residual Blocksを作成する関数を定義
    def _make_layer(self, block, channels, blocks, stride):
        layers = []
        # 最初のResidual Block（stride=stride）
        layers.append(block(self.in_channels, channels, stride))
        # 残りのResidual Block（stride=1）
        self.in_channels = channels * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.in_channels, channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

In [12]:
# モデルを呼び出す関数を定義
def supermaskresnet34_10():
    return SupermaskResNet(SupermaskBasicBlock, [3, 4, 6, 3], num_classes=10)

## 学習と評価（100 epochs, init=kaiming_uniform）

### prune_rate=0.999

In [17]:
# 刈り込み率を設定
prune_rate = 0.999
# 初期化手法を設定
init = 'kaiming_uniform'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskresnet34_10().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar10):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskResNet34_CIFAR10_100epochs_pr0.999_uniform_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 14.63 %, Loss: 2.2420
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 19.432 %, Loss: 2.1871
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 20.874 %, Loss: 2.0738
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 23.206 %, Loss: 2.0111
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 23.78 %, Loss: 2.0222
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 24.242 %, Loss: 2.0279
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 24.826 %, Loss: 2.0385
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 26.12 %, Loss: 1.9896
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 26.742 %, Loss: 1.9340
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 28.292 %, Loss: 1.9472
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 28.748

In [18]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [19]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar10:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 32.72 %


In [20]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.995

In [13]:
# 刈り込み率を設定
prune_rate = 0.995
# 初期化手法を設定
init = 'kaiming_uniform'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskresnet34_10().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar10):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskResNet34_CIFAR10_100epochs_pr0.995_uniform_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 22.516 %, Loss: 1.8979
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 31.144 %, Loss: 1.8004
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 32.836 %, Loss: 1.7646
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 34.32 %, Loss: 1.7915
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 36.034 %, Loss: 1.7529
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 37.184 %, Loss: 1.7680
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 37.864 %, Loss: 1.5572
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 38.564 %, Loss: 1.6669
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 39.398 %, Loss: 1.7196
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 40.58 %, Loss: 1.6543
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 41.11

In [14]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [15]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar10:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 52.51 %


In [16]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.99

In [74]:
# 刈り込み率を設定
prune_rate = 0.99
# 初期化手法を設定
init = 'kaiming_uniform'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskresnet34_10().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar10):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskResNet34_CIFAR10_100epochs_pr0.99_uniform_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 25.774 %, Loss: 1.8510
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 35.306 %, Loss: 1.7755
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 36.758 %, Loss: 1.6933
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 38.71 %, Loss: 1.5763
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 40.312 %, Loss: 1.5478
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 41.164 %, Loss: 1.5465
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 42.63 %, Loss: 1.5247
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 43.908 %, Loss: 1.4645
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 44.334 %, Loss: 1.4686
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 45.28 %, Loss: 1.5027
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 45.522

In [75]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [76]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar10:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 50.7 %


In [77]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.95

In [54]:
# 刈り込み率を設定
prune_rate = 0.95
# 初期化手法を設定
init = 'kaiming_uniform'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskresnet34_10().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar10):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskResNet34_CIFAR10_100epochs_pr0.95_uniform_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 31.842 %, Loss: 1.6371
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 42.902 %, Loss: 1.4551
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 46.028 %, Loss: 1.3327
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 47.586 %, Loss: 1.4239
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 49.738 %, Loss: 1.2642
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 52.328 %, Loss: 1.2673
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 53.734 %, Loss: 1.3723
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 55.046 %, Loss: 1.1766
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 55.616 %, Loss: 1.1816
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 57.494 %, Loss: 1.0771
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 58.

In [55]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [56]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar10:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 72.59 %


In [57]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.9

In [18]:
# 刈り込み率を設定
prune_rate = 0.9
# 初期化手法を設定
init = 'kaiming_uniform'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskresnet34_10().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar10):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskResNet34_CIFAR10_100epochs_pr0.9_uniform_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 30.994 %, Loss: 1.6692
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 42.168 %, Loss: 1.3576
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 46.15 %, Loss: 1.4106
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 49.242 %, Loss: 1.2990
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 51.222 %, Loss: 1.3311
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 53.482 %, Loss: 1.2262
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 55.664 %, Loss: 1.3473
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 57.878 %, Loss: 1.2314
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 58.836 %, Loss: 1.0717
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 60.656 %, Loss: 1.1271
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 61.8

In [19]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [20]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar10:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 77.6 %


In [21]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.7

In [14]:
# 刈り込み率を設定
prune_rate = 0.7
# 初期化手法を設定
init = 'kaiming_uniform'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskresnet34_10().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar10):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskResNet34_CIFAR10_100epochs_pr0.7_uniform_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 30.148 %, Loss: 1.7142
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 41.308 %, Loss: 1.4656
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 45.278 %, Loss: 1.3623
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 49.492 %, Loss: 1.2605
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 53.056 %, Loss: 1.3890
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 55.706 %, Loss: 1.1623
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 58.13 %, Loss: 1.1180
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 60.278 %, Loss: 1.1922
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 62.114 %, Loss: 1.1275
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 63.598 %, Loss: 1.0936
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 64.6

In [15]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [16]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar10:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 81.18 %


In [17]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.5

In [38]:
# 刈り込み率を設定
prune_rate = 0.5
# 初期化手法を設定
init = 'kaiming_uniform'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskresnet34_10().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar10):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskResNet34_CIFAR10_100epochs_pr0.5_uniform_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 29.242 %, Loss: 1.7573
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 39.466 %, Loss: 1.5688
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 43.748 %, Loss: 1.5200
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 47.632 %, Loss: 1.3773
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 50.346 %, Loss: 1.3154
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 53.846 %, Loss: 1.2863
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 56.858 %, Loss: 1.2767
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 59.178 %, Loss: 1.0896
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 61.598 %, Loss: 1.0355
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 63.172 %, Loss: 1.0219
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 64.

In [39]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [40]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar10:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 80.24 %


In [41]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

## 学習と評価（100 epochs, init=kaiming_normal）

### prune_rate=0.999

In [25]:
# 刈り込み率を設定
prune_rate = 0.999
# 初期化手法を設定
init = 'kaiming_normal'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskresnet34_10().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar10):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskResNet34_CIFAR10_100epochs_pr0.999_normal_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 14.752 %, Loss: 2.2729
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 17.032 %, Loss: 2.1434
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 21.674 %, Loss: 2.0038
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 23.124 %, Loss: 2.0461
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 25.224 %, Loss: 2.0233
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 26.112 %, Loss: 1.8846
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 27.6 %, Loss: 1.9547
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 27.534 %, Loss: 2.0112
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 27.84 %, Loss: 1.9554
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 25.176 %, Loss: 2.0895
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 22.922

In [26]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [27]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar10:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 27.48 %


In [28]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.995

In [21]:
# 刈り込み率を設定
prune_rate = 0.995
# 初期化手法を設定
init = 'kaiming_normal'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskresnet34_10().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar10):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskResNet34_CIFAR10_100epochs_pr0.995_normal_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 19.686 %, Loss: 2.0046
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 29.798 %, Loss: 1.7936
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 30.668 %, Loss: 1.8188
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 32.95 %, Loss: 1.8047
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 33.996 %, Loss: 1.7821
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 34.938 %, Loss: 1.7045
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 36.108 %, Loss: 1.7217
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 36.508 %, Loss: 1.6282
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 37.816 %, Loss: 1.6760
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 38.18 %, Loss: 1.6673
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 38.57

In [22]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [23]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar10:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 46.21 %


In [24]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.99

In [70]:
# 刈り込み率を設定
prune_rate = 0.99
# 初期化手法を設定
init = 'kaiming_normal'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskresnet34_10().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar10):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskResNet34_CIFAR10_100epochs_pr0.99_normal_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 24.478 %, Loss: 1.8607
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 34.566 %, Loss: 1.6225
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 36.136 %, Loss: 1.6717
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 38.69 %, Loss: 1.5196
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 39.796 %, Loss: 1.6799
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 40.814 %, Loss: 1.6035
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 42.206 %, Loss: 1.5635
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 42.91 %, Loss: 1.5851
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 43.698 %, Loss: 1.5824
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 44.704 %, Loss: 1.5174
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 46.04

In [71]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [72]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar10:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 52.61 %


In [73]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.95

In [58]:
# 刈り込み率を設定
prune_rate = 0.95
# 初期化手法を設定
init = 'kaiming_normal'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskresnet34_10().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar10):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskResNet34_CIFAR10_100epochs_pr0.95_normal_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 30.128 %, Loss: 1.5618
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 40.918 %, Loss: 1.5662
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 43.944 %, Loss: 1.3070
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 47.022 %, Loss: 1.3802
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 48.634 %, Loss: 1.4721
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 50.742 %, Loss: 1.3380
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 51.822 %, Loss: 1.3149
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 53.64 %, Loss: 1.2977
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 54.166 %, Loss: 1.2265
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 55.168 %, Loss: 1.2869
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 56.6

In [59]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [60]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar10:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 67.79 %


In [61]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.9

In [22]:
# 刈り込み率を設定
prune_rate = 0.9
# 初期化手法を設定
init = 'kaiming_normal'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskresnet34_10().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar10):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskResNet34_CIFAR10_100epochs_pr0.9_normal_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 31.444 %, Loss: 1.6616
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 42.374 %, Loss: 1.4694
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 45.312 %, Loss: 1.4743
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 49.018 %, Loss: 1.3714
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 51.438 %, Loss: 1.2413
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 53.756 %, Loss: 1.3112
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 55.158 %, Loss: 1.2996
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 56.86 %, Loss: 1.2075
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 58.722 %, Loss: 1.0711
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 59.934 %, Loss: 1.1654
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 60.9

In [23]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [24]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar10:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 73.72 %


In [25]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.7

In [26]:
# 刈り込み率を設定
prune_rate = 0.7
# 初期化手法を設定
init = 'kaiming_normal'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskresnet34_10().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar10):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskResNet34_CIFAR10_100epochs_pr0.7_normal_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 29.312 %, Loss: 1.6773
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 40.196 %, Loss: 1.4896
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 44.17 %, Loss: 1.3333
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 48.244 %, Loss: 1.2690
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 51.724 %, Loss: 1.2757
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 54.118 %, Loss: 1.1035
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 56.694 %, Loss: 1.1127
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 58.802 %, Loss: 1.0501
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 60.586 %, Loss: 1.1406
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 62.736 %, Loss: 1.1344
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 63.8

In [27]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [28]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar10:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 79.83 %


In [29]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.5

In [42]:
# 刈り込み率を設定
prune_rate = 0.5
# 初期化手法を設定
init = 'kaiming_normal'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskresnet34_10().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar10):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskResNet34_CIFAR10_100epochs_pr0.5_normal_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 28.722 %, Loss: 1.7329
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 39.202 %, Loss: 1.5465
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 42.836 %, Loss: 1.5302
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 46.608 %, Loss: 1.3573
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 50.182 %, Loss: 1.3374
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 53.124 %, Loss: 1.1716
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 55.694 %, Loss: 1.1564
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 58.292 %, Loss: 1.2997
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 60.188 %, Loss: 1.2081
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 62.206 %, Loss: 1.0801
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 63.

In [43]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [44]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar10:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 79.47 %


In [45]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

## 学習と評価（100 epochs, init=signed_constant）

### prune_rate=0.999

In [35]:
# 刈り込み率を設定
prune_rate = 0.999
# 初期化手法を設定
init = 'signed_constant'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskresnet34_10().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar10):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskResNet34_CIFAR10_100epochs_pr0.999_constant_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 14.362 %, Loss: 2.2053
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 21.474 %, Loss: 2.0616
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 24.134 %, Loss: 2.0674
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 25.824 %, Loss: 1.9390
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 25.722 %, Loss: 1.8786
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 28.052 %, Loss: 1.9797
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 28.86 %, Loss: 1.8331
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 28.512 %, Loss: 1.8517
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 29.206 %, Loss: 1.8441
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 29.098 %, Loss: 1.8520
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 29.7

In [36]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [37]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar10:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 32.0 %


In [38]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.995

In [29]:
# 刈り込み率を設定
prune_rate = 0.995
# 初期化手法を設定
init = 'signed_constant'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskresnet34_10().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar10):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskResNet34_CIFAR10_100epochs_pr0.995_constant_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 22.144 %, Loss: 2.0142
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 33.32 %, Loss: 1.7987
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 34.652 %, Loss: 1.7582
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 36.898 %, Loss: 1.6560
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 39.26 %, Loss: 1.7206
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 39.732 %, Loss: 1.6785
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 40.606 %, Loss: 1.5499
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 41.23 %, Loss: 1.6525
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 42.31 %, Loss: 1.5783
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 43.308 %, Loss: 1.5417
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 43.956 

In [30]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [31]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar10:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 55.95 %


In [32]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.99

In [66]:
# 刈り込み率を設定
prune_rate = 0.99
# 初期化手法を設定
init = 'signed_constant'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskresnet34_10().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar10):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskResNet34_CIFAR10_100epochs_pr0.99_constant_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 28.394 %, Loss: 1.7259
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 38.536 %, Loss: 1.6229
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 40.168 %, Loss: 1.6608
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 41.69 %, Loss: 1.4530
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 43.56 %, Loss: 1.5370
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 45.032 %, Loss: 1.4524
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 46.044 %, Loss: 1.5185
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 47.59 %, Loss: 1.3622
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 48.084 %, Loss: 1.4444
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 48.592 %, Loss: 1.6619
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 50.516

In [67]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [68]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar10:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 59.31 %


In [69]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.95

In [62]:
# 刈り込み率を設定
prune_rate = 0.95
# 初期化手法を設定
init = 'signed_constant'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskresnet34_10().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar10):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskResNet34_CIFAR10_100epochs_pr0.95_constant_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 32.28 %, Loss: 1.5744
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 44.492 %, Loss: 1.4581
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 46.422 %, Loss: 1.3977
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 50.118 %, Loss: 1.3877
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 51.982 %, Loss: 1.2530
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 54.342 %, Loss: 1.2958
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 56.088 %, Loss: 1.2516
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 57.98 %, Loss: 1.1674
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 59.41 %, Loss: 1.1145
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 60.334 %, Loss: 1.0382
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 61.704

In [63]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [64]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar10:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 77.38 %


In [65]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.9

In [30]:
# 刈り込み率を設定
prune_rate = 0.9
# 初期化手法を設定
init = 'signed_constant'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskresnet34_10().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar10):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskResNet34_CIFAR10_100epochs_pr0.9_constant_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 32.962 %, Loss: 1.6122
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 44.364 %, Loss: 1.5083
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 47.64 %, Loss: 1.4347
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 50.75 %, Loss: 1.3389
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 54.008 %, Loss: 1.3309
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 56.16 %, Loss: 1.1209
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 58.656 %, Loss: 1.1514
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 60.674 %, Loss: 1.2014
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 61.972 %, Loss: 1.0197
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 63.138 %, Loss: 1.1064
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 64.456

In [31]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [32]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar10:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 80.79 %


In [33]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.7

In [34]:
# 刈り込み率を設定
prune_rate = 0.7
# 初期化手法を設定
init = 'signed_constant'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskresnet34_10().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar10):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskResNet34_CIFAR10_100epochs_pr0.7_constant_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 31.898 %, Loss: 1.5809
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 42.704 %, Loss: 1.5156
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 46.272 %, Loss: 1.3828
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 50.618 %, Loss: 1.4256
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 53.674 %, Loss: 1.2318
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 56.218 %, Loss: 1.1452
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 59.274 %, Loss: 1.1113
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 61.462 %, Loss: 1.1514
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 63.316 %, Loss: 1.0247
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 64.908 %, Loss: 0.9311
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 66.

In [35]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [36]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar10:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 82.51 %


In [49]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.5

In [50]:
# 刈り込み率を設定
prune_rate = 0.5
# 初期化手法を設定
init = 'signed_constant'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskresnet34_10().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar10):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskResNet34_CIFAR10_100epochs_pr0.5_constant_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 28.714 %, Loss: 1.7926
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 39.75 %, Loss: 1.5991
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 44.104 %, Loss: 1.4456
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 48.568 %, Loss: 1.3306
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 52.39 %, Loss: 1.1516
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 55.512 %, Loss: 1.1383
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 58.142 %, Loss: 1.1564
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 60.56 %, Loss: 1.0561
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 62.852 %, Loss: 1.0406
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 64.73 %, Loss: 0.9868
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 65.89 %

In [51]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [52]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar10:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 80.8 %


In [53]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()